In [ ]:
#Establecer conexion con la página
import urllib.request
#Leer y decodificar la página
datos = urllib.request.urlopen('https://www.dof.gob.mx/index_111.php?year=2022&month=08&day=30#gsc.tab=0').read().decode()

In [ ]:
datos

In [ ]:
#Instalar libreria
import sys
!{sys.executable} -m pip install mysql-connector-python==8.0.29

#librerias: requests, bs4, mysql-connector-python, lxml

In [2]:
#Transformamos todo el HTML en un objeto de Beautiful Soap
from msilib.schema import Property
from bs4 import BeautifulSoup
import re 
soup = BeautifulSoup(datos, 'lxml')
#Búsqueda de todas las etiquetas tr
box = soup.find('tr',id="trcontent")
#Búsqueda de las etiquetas a
resultado = box.find_all('a',class_='enlaces')
#Búsqueda de la palabra clave
palabra = re.findall(r'69-B', str(resultado))
print(palabra)


#Contar elementos
transforma = ", ".join(map(str, palabra))
conteo = len(transforma.split())
#print("Hay " + str(conteo) + " palabras encontradas.")


[]


In [3]:

#Recupera todos los link de las publicaciones y las almacena en una lista -links[]
lst_links =[]
for ref in resultado:
      tag=ref.get('href')+'\n'
      transforma="".join(map(str, tag))
      link=[transforma.split('\n')]
      lst_links.append(link)
      

#Resguarda especialmente los links en donde se encuentre la palabra 69-B en una lista -lst_69B[]
lst_69B =[]
x=0
for Pos_info in resultado:
    
    encuentra = re.findall(r'69-B', str(Pos_info))
    if bool(encuentra)==True:
        #print(encuentra)
        #print(resultado[x])
        #print(links[x])
        PosLink = lst_links[x]
        lst_69B.append(PosLink)
    
    x+=1
print (lst_69B)


[]


In [4]:
#Elimina carcateres innecesarios de los links y las concatena con el dominio de la pagina 
lst_link_dominio=[]
for Textlink in lst_69B:
  cadena = Textlink[0]
  characters = "[]'"

  cadena = ''.join( x for x in cadena if x not in characters)
  cal='https://www.dof.gob.mx'+cadena
  lst_link_dominio.append(cal)
  # print(cal)

# print(lst_link_dominio)
str_links = ", ".join(lst_link_dominio)
print(str_links)

In [5]:
#importamos para envios de correos 
import smtplib 
from email.message import EmailMessage 

if bool(palabra) == True:
    asunto_correo = "Publicaciones del Diario Oficial de la Federación" 
    #correo_receptor = "frester_dui98@hotmail.com" 
    correo_remitente = "jnahuat@nasa.com.mx" 
    correo_receptor = "frester_dui98@hotmail.com"
    email_smtp = "smtp.office365.com" 
    contrasena_correo = "stckrhhmhjxfqwgl" #contraseña aplicacion Nasa
    #email_password = "poykyzptkdiihjhe" #contraseña aplicacion hotmail frester
 

    
    mensaje = EmailMessage() 

    # Configuracion de los titulos 
    mensaje['Subject'] = asunto_correo 
    mensaje['From'] = correo_remitente 
    mensaje['To'] = correo_receptor

    # Mensaje de texto que se enviará
    mensaje.set_content("Se ha identificado una nueva publicación referente al artículo 69-B: \n"+str_links) 

    try:
        # smtp servidor y puerto 
        server = smtplib.SMTP('smtp.office365.com', port=587) 

        # Identificar este cliente en el servidor 
        server.ehlo() 

        # iniciar conexion del servidor SMTP
        server.starttls() 

        # Iniciar sesion con el correo electronico
        server.login(correo_remitente, contrasena_correo) 

        # se envia el mensaje por correo 
        server.send_message(mensaje) 

        # se cierra la conexion del servidor 
        server.quit()

        print('Correo electronico enviado con exito')
        
    except smtplib.SMTPException as e:
        print ('Error al intentar enviar el correo electronico, cuasa del error: ', e)

if bool(palabra) == False:
    asunto_correo = "Publicaciones del Diario Oficial de la Federación" 
    #correo_receptor = "frester_dui98@hotmail.com" 
    correo_remitente = "jnahuat@nasa.com.mx" 
    correo_receptor = "frester_dui98@hotmail.com"
    email_smtp = "smtp.office365.com" 
    contrasena_correo = "stckrhhmhjxfqwgl" #contraseña aplicacion Nasa
    #email_password = "poykyzptkdiihjhe" #contraseña aplicacion hotmail frester
 

    
    mensaje = EmailMessage() 

    # Configuracion de los titulos
    mensaje['Subject'] = asunto_correo 
    mensaje['From'] = correo_remitente 
    mensaje['To'] = correo_receptor

    # Mensaje que se le envia al correo
    mensaje.set_content("No se ha encontrado publicación nueva referente al articulo 69-B") 

    try:
        # smtp servidor y puerto 
        server = smtplib.SMTP('smtp.office365.com', port=587) 

        # Identificar este cliente en el servidor 
        server.ehlo() 

        # iniciar conexion del servidor SMTP
        server.starttls() 

        # Iniciar sesion con el correo electronico
        server.login(correo_remitente, contrasena_correo) 

        # se envia el mensaje por correo 
        server.send_message(mensaje) 

        # se cierra la conexion del servidor 
        server.quit()

        print('Correo electronico enviado con exito')

    except smtplib.SMTPException as e:
        print ('Error al intentar enviar el correo electronico, cuasa del error: ', e)

Correo electronico enviado con exito


In [20]:
#Conexión del programa con la base de datos
import mysql.connector

try:
    connection=mysql.connector.connect(
        host='localhost',
        port='3306',
        user='root',
        db='scraper'
    )

    if connection.is_connected():
        print('Conexión exitosa')

except Exception as ex:
    print(ex)

Conexión exitosa


In [21]:
#Comprobación de conexión a la base de datos
cursor = connection.cursor()
cursor.execute('SELECT DATABASE()')
row = cursor.fetchone()
print('Conectado a la base de datos: {}'.format(row))

Conectado a la base de datos: ('scraper',)


In [22]:
#Obtener fecha actual
from datetime import datetime

today = datetime.now()
print(today)


2022-10-10 17:27:55.955607


In [23]:
#Insercion de valores en la base de datos
for ciclo in range(conteo):
    cursor.execute("INSERT INTO `extraccion` (`id`, `link`, `fecha`) VALUES (NULL,%(liga)s, %(fecha_actual)s)",{"liga":cal,"fecha_actual":today})

#Consulta de datos
cursor.execute("SELECT * FROM `extraccion`")
valores = cursor.fetchall()

#Impresión de datos
for fila in valores:
    print(fila)
    
#Envío de los valores a la base de datos
connection.commit()

(1, '69-B', datetime.datetime(2022, 10, 4, 11, 8, 23))
(2, '69-B', datetime.datetime(2022, 10, 4, 11, 8, 23))
(3, 'https://www.dof.gob.mx/nota_detalle.php?codigo=5665409&fecha=26/09/2022', datetime.datetime(2022, 10, 6, 13, 2, 12))
(4, 'https://www.dof.gob.mx/nota_detalle.php?codigo=5665409&fecha=26/09/2022', datetime.datetime(2022, 10, 6, 13, 2, 12))
(5, 'https://www.dof.gob.mx/nota_detalle.php?codigo=5667636&fecha=10/10/2022', datetime.datetime(2022, 10, 10, 16, 37, 12))
(6, 'https://www.dof.gob.mx/nota_detalle.php?codigo=5667636&fecha=10/10/2022', datetime.datetime(2022, 10, 10, 16, 37, 12))
(7, 'https://www.dof.gob.mx/nota_detalle.php?codigo=5667636&fecha=10/10/2022', datetime.datetime(2022, 10, 10, 17, 27, 56))
(8, 'https://www.dof.gob.mx/nota_detalle.php?codigo=5667636&fecha=10/10/2022', datetime.datetime(2022, 10, 10, 17, 27, 56))


In [24]:
#Cerrar la conexión
connection.close()
print('Conexión cerrada exitosamente')


Conexión cerrada exitosamente
